## PySpark训练word2vec实现内容相似推荐

实现步骤：
1. 获取文章列表数据，包括ID、标题、内容
2. 使用jieba实现中文分词
3. 送入pyspark实现word2vec的训练，得到文章向量
4. 对于输入的ID，计算最相似的文章列表

### 1. 获取数据

In [1]:
import pandas as pd

In [3]:
csv_file = "./datas/ZhengFeixiang.csv"
csv_data = pd.read_csv(csv_file, low_memory = False)#防止弹出警告
df = pd.DataFrame(csv_data)

In [4]:
df.head(3)

,名字,账号,试卷,试题,对0错1,时间,详情,试卷名称,试题简介,章节
0,郑飞翔,202216012,5695,46499,0,20-09-07 10:27:23,e6d7733789671e1ea1ce5ff38cda6558,第1节 长度和时间的测量,(柳州中考)一个中学生的身高约为(),NaN
1,郑飞翔,202216012,5695,46500,0,20-09-07 10:27:24,be7b44771617bbdda1ce5ff38cda6558,第1节 长度和时间的测量,下列长度单位换算正确的是(),NaN
2,郑飞翔,202216012,5695,46502,1,20-09-07 10:27:25,f7b4315417d11ef3a1ce5ff38cda6558,第1节 长度和时间的测量,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()",NaN


### 2. 使用jieba实现中文分词

pip install jieba

In [5]:
import jieba

In [6]:
def do_cut_words(param_df):
    # 标题加上关键词，是整个待分词的句子
    sentence = param_df["试卷名称"]+","+(param_df["试题简介"])
    # 调用分词
    words = list(jieba.cut(sentence))
    # 做过滤，变成小写
    result = []
    for word in words:
        if not word or len(word)==0 or len(word)==1: 
            continue
        word = word.lower()
        result.append(word)
    return " ".join(result)

df["words"] = df.apply(do_cut_words, axis=1)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\2\jieba.cache
Loading model cost 1.506 seconds.
Prefix dict has been built successfully.


In [8]:
#添加自增的序号列
df['id'] = range(len(df))

In [9]:
df.head(5)

,名字,账号,试卷,试题,对0错1,时间,详情,试卷名称,试题简介,章节,words,id
0,郑飞翔,202216012,5695,46499,0,20-09-07 10:27:23,e6d7733789671e1ea1ce5ff38cda6558,第1节 长度和时间的测量,(柳州中考)一个中学生的身高约为(),NaN,长度 时间 测量 柳州 中考 一个 中学生 身高,0
1,郑飞翔,202216012,5695,46500,0,20-09-07 10:27:24,be7b44771617bbdda1ce5ff38cda6558,第1节 长度和时间的测量,下列长度单位换算正确的是(),NaN,长度 时间 测量 下列 长度 单位 换算 正确,1
2,郑飞翔,202216012,5695,46502,1,20-09-07 10:27:25,f7b4315417d11ef3a1ce5ff38cda6558,第1节 长度和时间的测量,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()",NaN,长度 时间 测量 初中生 小红想 对比 测量 一个月 前后 自己 身高 变化 选用 下列 哪...,2
3,郑飞翔,202216012,5695,46503,0,20-09-07 10:27:25,b9cb1cfbf6353064a1ce5ff38cda6558,第1节 长度和时间的测量,如图所示测量木块长度的方法中正确的是(),NaN,长度 时间 测量 如图所示 测量 木块 长度 方法 正确,3
4,郑飞翔,202216012,5695,46505,1,20-09-07 10:27:25,8e3e6b329f6b4ab3a1ce5ff38cda6558,第1节 长度和时间的测量,下列各个过程中经历的时间最接近1秒的是(),NaN,长度 时间 测量 下列 各个 过程 经历 时间 接近,4


In [11]:
# 保存成CSV
df[["id", "试卷名称","试题简介", "words"]].to_csv("./datas/questions_wordsegs.csv", index=False)

### 3. 使用pyspark训练word2vec

In [12]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("test pyspark") \
    .getOrCreate()

sc = spark.sparkContext

#### Pyspark读取CSV数据

In [13]:
df = spark.read.csv("./datas/questions_wordsegs.csv", header=True)
df.show(5)

+---+-----------------------+--------------------------------------+--------------------------------+
| id|               试卷名称|                              试题简介|                           words|
+---+-----------------------+--------------------------------------+--------------------------------+
|  0|第1节　长度和时间的测量|      (柳州中考)一个中学生的身高约为()|长度 时间 测量 柳州 中考 一个...|
|  1|第1节　长度和时间的测量|            下列长度单位换算正确的是()|长度 时间 测量 下列 长度 单位...|
|  2|第1节　长度和时间的测量| 初中生小红想对比测量一个月前后自己...|长度 时间 测量 初中生 小红想 ...|
|  3|第1节　长度和时间的测量|如图所示测量木块长度的方法中正确的是()|长度 时间 测量 如图所示 测量 ...|
|  4|第1节　长度和时间的测量|  下列各个过程中经历的时间最接近1秒...|长度 时间 测量 下列 各个 过程...|
+---+-----------------------+--------------------------------------+--------------------------------+
only showing top 5 rows



In [14]:
from pyspark.sql import functions as F
from pyspark.sql import types as T

In [15]:
# 把非常的字符串格式变成LIST形式
df = df.withColumn('words_split', F.split(df.words, " "))

#### 实现word2vec的训练与转换

In [16]:
# https://spark.apache.org/docs/2.4.6/ml-features.html#word2vec

from pyspark.ml.feature import Word2Vec

word2Vec = Word2Vec(
    vectorSize=5, 
    minCount=0, 
    inputCol="words_split", 
    outputCol="word2vec")

model = word2Vec.fit(df)

# 注意这一步，会得到整个doc的word embedding
df_word2vec = model.transform(df)

In [17]:
df_word2vec.printSchema()

root
 |-- id: string (nullable = true)
 |-- 试卷名称: string (nullable = true)
 |-- 试题简介: string (nullable = true)
 |-- words: string (nullable = true)
 |-- words_split: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- word2vec: vector (nullable = true)



In [18]:
df_word2vec.select("word2vec").show(3, truncate=False)

+------------------------------------------------------------------------------------------------------------+
|word2vec                                                                                                    |
+------------------------------------------------------------------------------------------------------------+
|[0.030937873758375645,-0.20446800347417593,0.07760335924103856,0.05660174344666302,0.021415553288534284]    |
|[-0.07923593744635582,-0.26820977265015244,-0.04264575755223632,-0.14990293351002038,0.08549118041992188]   |
|[-0.008039933629333973,-0.20343961799517274,-0.061635409540031105,0.011067528743296862,0.048510019201785326]|
+------------------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [22]:
df_word2vec.select("id", "试卷名称","试题简介", "word2vec") \
           .toPandas() \
           .to_csv('./datas/questions_words_word2vec.csv', index=False)

### 4. 对于给定文章算出最相似的10篇文章

In [23]:
df = pd.read_csv("./datas/questions_words_word2vec.csv")
df.head(3)

,id,试卷名称,试题简介,word2vec
0,0,第1节 长度和时间的测量,(柳州中考)一个中学生的身高约为(),"[0.030937873758375645,-0.20446800347417593,0.0..."
1,1,第1节 长度和时间的测量,下列长度单位换算正确的是(),"[-0.07923593744635582,-0.26820977265015244,-0...."
2,2,第1节 长度和时间的测量,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()","[-0.008039933629333973,-0.20343961799517274,-0..."


In [24]:
import numpy as np
import json

In [25]:
df["word2vec"] = df["word2vec"].map(lambda x : np.array(json.loads(x)))

In [26]:
df.head(3)

,id,试卷名称,试题简介,word2vec
0,0,第1节 长度和时间的测量,(柳州中考)一个中学生的身高约为(),"[0.030937873758375645, -0.20446800347417593, 0..."
1,1,第1节 长度和时间的测量,下列长度单位换算正确的是(),"[-0.07923593744635582, -0.26820977265015244, -..."
2,2,第1节 长度和时间的测量,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()","[-0.008039933629333973, -0.20343961799517274, ..."


In [27]:
# 随便挑选一篇文章ID，2583：pandas，581：PHP
article_id = 581
df.loc[df["id"]==article_id]

,id,试卷名称,试题简介,word2vec
581,581,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0...."


In [28]:
article_embedding = df.loc[df["id"]==article_id, "word2vec"].iloc[0]
article_embedding

array([-0.28099852,  0.40256441, -0.37525716, -0.02898862,  0.63318622])

In [29]:
# 余弦相似度
from scipy.spatial import distance
df["sim_value"] = df["word2vec"].map(lambda x : 1 - distance.cosine(article_embedding, x))

In [32]:
df[["id", "试题简介", "sim_value"]].head(10)

,id,试题简介,sim_value
0,0,(柳州中考)一个中学生的身高约为(),-0.533752
1,1,下列长度单位换算正确的是(),-0.038270
2,2,"初中生小红想对比测量一个月前后自己的身高变化,选用下列哪种尺最合适()",-0.135035
3,3,如图所示测量木块长度的方法中正确的是(),-0.290644
4,4,下列各个过程中经历的时间最接近1秒的是(),-0.125849
5,5,"物理实验室中,常用的测量时间的工具是()",-0.555954
6,6,"图为小组同学学习长度测量后讨论交流的情境,下列关于误差说法中正确的是()",0.514104
7,7,"在测量物体长度时,由于下列原因造成测量结果有差异,其中属于误差的是()",-0.373059
8,8,下列估计的数据与实际最接近的是(),-0.068386
9,9,"某同学测得物理课本长为25.91 cm,宽为18.35 cm,那么他所用刻度尺的分度值为()",-0.351328


In [34]:
# 按相似度降序排列，查询前10条
df.sort_values(by="sim_value", ascending=False).head(20)

,id,试卷名称,试题简介,word2vec,sim_value
581,581,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
444,444,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
407,407,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
516,516,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
527,527,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
545,545,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
594,594,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
617,617,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
730,730,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
401,401,-,(2014武汉中考)下列关于声的说法正确的是,"[-0.28099851975483553, 0.4025644143777234, -0....",1.000000
